In [1]:
from torch_geometric.nn import HypergraphConv
import math
import torch
import torch.nn as nn 
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import torch.nn.init as init
# from torch.nn.init import xavier_normal_, xavier_uniform_
import argparse
import numpy as np
import time
import sys
from os.path import abspath
import random
import pandas as pd
from util.sampler import  next_batch_pairwise
from util.conf import OptionConf
import torch
import torch.nn as nn 
import torch.nn.functional as F
from scipy.sparse import coo_matrix
from util.loss_torch import bpr_loss, l2_reg_loss, EmbLoss, contrastLoss, InfoNCE
from util.init import *
from base.torch_interface import TorchGraphInterface
import os
import numpy as np 
import time 
from torch.optim.lr_scheduler import ReduceLROnPlateau
from itertools import product

from util.conf import ModelConf
from base.recommender import Recommender
from util.algorithm import find_k_largest
from time import strftime, localtime
from data.loader import FileIO
from util.evaluation import ranking_evaluation
from util.evaluation import early_stopping

from data.ui_graph import Interaction
from data.augmentor import GraphAugmentor, drop_edges
from torch_geometric.nn import HypergraphConv

/home/aiotlab3/anaconda3/envs/hungvv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cuda', index=0)

## Base Recommender

In [4]:
class GraphRecommender(Recommender):
    def __init__(self, conf, data, data_kg,knowledge_set, **kwargs):
        super(GraphRecommender, self).__init__(conf, data, data_kg, knowledge_set,**kwargs)
        self.data = data
        self.bestPerformance = []
        top = self.ranking['-topN'].split(',')
        self.topN = [int(num) for num in top]
        self.max_N = max(self.topN)
        
        self.dataset = kwargs['dataset']
        
        # self.output = f"./results/{self.model_name}/{self.dataset}/@{self.model_name}-inp_emb:{kwargs['input_dim']}-hyper_emb:{kwargs['hyper_dim']}-bs:{self.batch_size}-lr:{kwargs['lr']}-lrd:{kwargs['lr_decay']}-reg:{kwargs['reg']}-leaky:{kwargs['p']}-dropout:{kwargs['drop_rate']}-n_layers:{kwargs['n_layers']}-n_heads:{kwargs['n_heads']}-n_self_att:{kwargs['n_self_att']}/"
        self.output = f"./results/HGNN_Attention_DropEdge/{self.dataset}/@{self.model_name}-inp_emb:{kwargs['input_dim']}-hyper_emb:{kwargs['hyper_dim']}-bs:{self.batch_size}-opt:{kwargs['optimizer']}-lr:{kwargs['lr']}-lrd:{kwargs['lr_decay']}-reg:{kwargs['reg']}-leaky:{kwargs['p']}-dropout:{kwargs['drop_rate']}-concat:{kwargs['concat']}-attention_mode:{kwargs['attention_mode']}-n_layers:{kwargs['n_layers']}-cl_rate:{kwargs['cl_rate']}-aug_type:{kwargs['aug_type']}-temp:{kwargs['temp']}-wdecay:{kwargs['weight_decay']}/"
        if not os.path.exists(self.output):
            os.makedirs(self.output)

    def print_model_info(self):
        super(GraphRecommender, self).print_model_info()
        # # print dataset statistics
        print('Training Set Size: (user number: %d, item number %d, interaction number: %d)' % (self.data.training_size()))
        print('Test Set Size: (user number: %d, item number %d, interaction number: %d)' % (self.data.test_size()))
        print('=' * 80)

    def build(self):
        pass

    def train(self):
        pass

    def predict(self, u):
        pass

    def test(self, user_emb, item_emb):
        def process_bar(num, total):
            rate = float(num) / total
            ratenum = int(50 * rate)
            r = '\rProgress: [{}{}]{}%'.format('+' * ratenum, ' ' * (50 - ratenum), ratenum*2)
            sys.stdout.write(r)
            sys.stdout.flush()

        # predict
        rec_list = {}
        user_count = len(self.data.test_set)
        for i, user in enumerate(self.data.test_set):
            # s_find_candidates = time.time()
            
            
            # candidates = predict(user)
            user_id  = self.data.get_user_id(user)
            score = torch.matmul(user_emb[user_id], item_emb.transpose(0, 1))
            candidates = score.cpu().numpy()
            
            # e_find_candidates = time.time()
            # print("Calculate candidates time: %f s" % (e_find_candidates - s_find_candidates))
            # predictedItems = denormalize(predictedItems, self.data.rScale[-1], self.data.rScale[0])
            rated_list, li = self.data.user_rated(user)
            for item in rated_list:
                candidates[self.data.item[item]] = -10e8
            
            # s_find_k_largest = time.time()
            ids, scores = find_k_largest(self.max_N, candidates)
            # e_find_k_largest = time.time()
            # print("Find k largest candidates: %f s" % (e_find_k_largest - s_find_k_largest))
            item_names = [self.data.id2item[iid] for iid in ids]
            rec_list[user] = list(zip(item_names, scores))
            if i % 1000 == 0:
                process_bar(i, user_count)
        process_bar(user_count, user_count)
        print('')
        return rec_list

    def evaluate(self, rec_list):
        self.recOutput.append('userId: recommendations in (itemId, ranking score) pairs, * means the item is hit.\n')
        for user in self.data.test_set:
            line = str(user) + ':'
            for item in rec_list[user]:
                line += ' (' + str(item[0]) + ',' + str(item[1]) + ')'
                if item[0] in self.data.test_set[user]:
                    line += '*'
            line += '\n'
            self.recOutput.append(line)
        current_time = strftime("%Y-%m-%d %H-%M-%S", localtime(time.time()))
        # output prediction result
        out_dir = self.output
        file_name = self.config['model.name'] + '@' + current_time + '-top-' + str(self.max_N) + 'items' + '.txt'
        FileIO.write_file(out_dir, file_name, self.recOutput)
        print('The result has been output to ', abspath(out_dir), '.')
        file_name = self.config['model.name'] + '@' + current_time + '-performance' + '.txt'
        self.result = ranking_evaluation(self.data.test_set, rec_list, self.topN)
        self.model_log.add('###Evaluation Results###')
        self.model_log.add(self.result)
        FileIO.write_file(out_dir, file_name, self.result)
        print('The result of %s:\n%s' % (self.model_name, ''.join(self.result)))

    def fast_evaluation(self, model, epoch, user_embed, item_embed, kwargs=None):
        print('Evaluating the model...')
        s_test = time.time()
        rec_list = self.test(user_embed, item_embed)
        e_test = time.time() 
        print("Test time: %f s" % (e_test - s_test))
        
        s_measure = time.time()
        measure = ranking_evaluation(self.data.test_set, rec_list, [self.max_N])
        e_measure = time.time()
        print("Measure time: %f s" % (e_measure - s_measure))
        
        if len(self.bestPerformance) > 0:
            count = 0
            performance = {}
            for m in measure[1:]:
                k, v = m.strip().split(':')
                performance[k] = float(v)
            for k in self.bestPerformance[1]:
                if self.bestPerformance[1][k] > performance[k]:
                    count += 1
                else:
                    count -= 1
            if count < 0:
                self.bestPerformance[1] = performance
                self.bestPerformance[0] = epoch + 1
                # try:
                #     self.save(kwargs)
                # except:
                self.save(model)
        else:
            self.bestPerformance.append(epoch + 1)
            performance = {}
            for m in measure[1:]:
                k, v = m.strip().split(':')
                performance[k] = float(v)
            self.bestPerformance.append(performance)
            # try:
            #     self.save(kwargs)
            # except:
            self.save(model)
        print('-' * 120)
        print('Real-Time Ranking Performance ' + ' (Top-' + str(self.max_N) + ' Item Recommendation)')
        measure = [m.strip() for m in measure[1:]]
        print('*Current Performance*')
        print('Epoch:', str(epoch + 1) + ',', '  |  '.join(measure))
        bp = ''
        # for k in self.bestPerformance[1]:
        #     bp+=k+':'+str(self.bestPerformance[1][k])+' | '
        bp += 'Hit Ratio' + ':' + str(self.bestPerformance[1]['Hit Ratio']) + '  |  '
        bp += 'Precision' + ':' + str(self.bestPerformance[1]['Precision']) + '  |  '
        bp += 'Recall' + ':' + str(self.bestPerformance[1]['Recall']) + '  |  '
        # bp += 'F1' + ':' + str(self.bestPerformance[1]['F1']) + ' | '
        bp += 'NDCG' + ':' + str(self.bestPerformance[1]['NDCG'])
        print('*Best Performance* ')
        print('Epoch:fast_evaluation', str(self.bestPerformance[0]) + ',', bp)
        print('-' * 120)
        return measure
    
    def save(self, model):
        with torch.no_grad():
            self.best_user_emb, self.best_item_emb = model()
        self.save_model(model)
    
    def save_model(self, model):
        # save model 
        current_time = strftime("%Y-%m-%d", localtime(time.time()))
        out_dir = self.output
        file_name =  self.config['model.name'] + '@' + current_time + '-weight' + '.pth'
        weight_file = out_dir + '/' + file_name 
        torch.save(model.state_dict(), weight_file)

    def save_loss(self, train_losses, rec_losses, reg_losses, cl_losses):
        df_train_loss = pd.DataFrame(train_losses, columns = ['ep', 'loss'])
        df_rec_loss = pd.DataFrame(rec_losses, columns = ['ep', 'loss'])
        df_reg_loss = pd.DataFrame(reg_losses, columns = ['ep', 'loss'])
        df_cl_loss =  pd.DataFrame(cl_losses, columns = ['ep', 'loss'])
        df_train_loss.to_csv(self.output + '/train_loss.csv')
        df_rec_loss.to_csv(self.output + '/rec_loss.csv')
        df_reg_loss.to_csv(self.output + '/reg_loss.csv')
        df_cl_loss.to_csv(self.output + '/cl_loss.csv')

    def save_perfomance_training(self, log_train):
        df_train_log = pd.DataFrame(log_train)
        df_train_log.to_csv(self.output + '/train_performance.csv')

## Layers

In [5]:
class BaseHyperGNN(torch.nn.Module):
    def __init__(self, dropout, p, emb_dim, num_layers, use_attention, attention_mode, heads, use_norm=True, use_skip_connection=True):
        super(BaseHyperGNN, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.dropout = dropout
        self.num_layers = num_layers
        self.ln = [nn.LayerNorm(emb_dim ).to(device) for i in range(num_layers)]
        self.act = nn.LeakyReLU(p)
        self.attention_mode = attention_mode
        self.use_norm = use_norm
        self.use_skip_connection = use_skip_connection
    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t, hyperedge_attr=None):
        if self.attention_mode:
            if self.use_norm:
                for i, conv in enumerate(self.convs[:-1]):
                    if self.use_skip_connection:
                        x = self.ln[i](conv(x, adj_t, hyperedge_attr=hyperedge_attr)) + x
                    else:
                        x = self.ln[i](conv(x, adj_t, hyperedge_attr=hyperedge_attr))
                    x = self.act(x)
                    x = F.dropout(x, p=self.dropout, training=self.training)
                if self.use_skip_connection:
                    x = self.convs[-1](x, adj_t, hyperedge_attr=hyperedge_attr) + x
                else:
                    x = self.convs[-1](x, adj_t, hyperedge_attr=hyperedge_attr)
                if self.num_layers == 1:
                    if self.use_skip_connection:
                        x = self.act(self.ln[0](self.convs[0](x, adj_t, hyperedge_attr=hyperedge_attr) + x) )
                    else:
                        x = self.act(self.ln[0](self.convs[0](x, adj_t, hyperedge_attr=hyperedge_attr)) )
                    x = F.dropout(x, p=self.dropout, training=self.training)
            else:
                for i, conv in enumerate(self.convs[:-1]):
                    if self.use_skip_connection:
                        x = conv(x, adj_t, hyperedge_attr=hyperedge_attr) + x
                    else:
                        x = conv(x, adj_t, hyperedge_attr=hyperedge_attr)
                    x = self.act(x)
                    x = F.dropout(x, p=self.dropout, training=self.training)
                if self.use_skip_connection:
                    x = self.convs[-1](x, adj_t, hyperedge_attr=hyperedge_attr) + x
                else:
                    x = self.convs[-1](x, adj_t, hyperedge_attr=hyperedge_attr)
                if self.num_layers == 1:
                    if self.use_skip_connection:
                        x = self.act(self.convs[0](x, adj_t, hyperedge_attr=hyperedge_attr) + x)
                    else:
                        x = self.act(self.convs[0](x, adj_t, hyperedge_attr=hyperedge_attr))
                    x = F.dropout(x, p=self.dropout, training=self.training)
        else:
            if self.use_norm:
                for i, conv in enumerate(self.convs[:-1]):
                    if self.use_skip_connection:
                        x = self.ln[i](conv(x, adj_t)) + x
                    else:
                        x = self.ln[i](conv(x, adj_t))
                    x = self.act(x)
                    x = F.dropout(x, p=self.dropout, training=self.training)
                if self.use_skip_connection:
                    x = self.convs[-1](x, adj_t) + x
                else:
                    x = self.convs[-1](x, adj_t)
                if self.num_layers == 1:
                    if self.use_skip_connection:
                        x = self.act(self.ln[0](self.convs[0](x, adj_t) + x) )
                    else:
                        x = self.act(self.ln[0](self.convs[0](x, adj_t)))
                    x = F.dropout(x, p=self.dropout, training=self.training)
            else:
                for i, conv in enumerate(self.convs[:-1]):
                    if self.use_skip_connection:
                        x = conv(x, adj_t) + x
                    else:
                        x = conv(x, adj_t)
                    x = self.act(x)
                    x = F.dropout(x, p=self.dropout, training=self.training)
                if self.use_skip_connection:
                    x = self.convs[-1](x, adj_t) + x
                else:
                    x = self.convs[-1](x, adj_t)
                if self.num_layers == 1:
                    if self.use_skip_connection:
                        x = self.act(self.convs[0](x, adj_t) + x )
                    else:
                        x = self.act(self.convs[0](x, adj_t))
                    x = F.dropout(x, p=self.dropout, training=self.training)
        return x


class HyperGCN(BaseHyperGNN):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers, dropout, p, use_attention, attention_mode, heads, concat, use_norm, use_skip_connection):
        super(HyperGCN, self).__init__(dropout, p, hidden_channels, num_layers, use_attention, attention_mode, heads, use_norm, use_skip_connection)

        for i in range(num_layers):
            first_channels = in_channels if i == 0 else hidden_channels
            second_channels = out_channels if i == num_layers - 1 else hidden_channels
            self.convs.append(HypergraphConv(first_channels, second_channels, use_attention=use_attention, attention_mode=attention_mode, heads=heads, concat=concat, negative_slove=p, dropout=dropout))

## Model

In [6]:
class Model(nn.Module):
    def __init__(self, config, data, args):
        super(Model, self).__init__()
        self.data = data
        self._parse_args(args)

        self.edge_index = data.edge_index.to(device)
        self.edge_index_t  = data.edge_index_t.to(device)

        if self.use_drop_edge:
            self.edge_index_aug = drop_edges(self.edge_index)
            self.edge_index_aug_t = drop_edges(self.edge_index_t)
        
        # adj = data.interaction_mat
        # self.adj  = TorchGraphInterface.convert_sparse_mat_to_tensor(adj).to(device)
        # self.sparse_norm_adj = TorchGraphInterface.convert_sparse_mat_to_tensor(data.norm_adj).to(device)
        
        self.embedding_dict = self._init_model()
        
        self.hgnn_u = HyperGCN(self.hyper_dim, self.hyper_dim, self.hyper_dim, self.layers, self.drop_rate, self.p, use_attention=True, attention_mode=args['attention_mode'], heads=args['n_heads'], concat=args['concat'], use_norm=args['use_norm'], use_skip_connection=args['use_skip_connection']) 
        self.hgnn_i = HyperGCN(self.hyper_dim, self.hyper_dim, self.hyper_dim, self.layers, self.drop_rate, self.p, use_attention=True, attention_mode=args['attention_mode'], heads=args['n_heads'], concat=args['concat'], use_norm=args['use_norm'], use_skip_connection=args['use_skip_connection']) 
        
        self.fc_u = nn.Linear(self.input_dim, self.hyper_dim)
        self.fc_i = nn.Linear(self.input_dim, self.hyper_dim)
        
        self.non_linear = nn.ReLU()
        self.act = nn.LeakyReLU(self.p)
        self.dropout = nn.Dropout(self.drop_rate)
        self.apply(self._init_weights)
        
    def _parse_args(self, args):
        self.input_dim = args['input_dim']
        self.hyper_dim = args['hyper_dim']
        self.p = args['p']
        self.drop_rate = args['drop_rate'] 
        self.layers = args['n_layers']
        self.temp = args['temp']
        self.aug_type = args['aug_type']
        self.use_drop_edge = args['use_drop_edge']

    def _init_model(self):
        initializer = init.xavier_uniform_
        embedding_dict = nn.ParameterDict({
            'user_emb': nn.Parameter(initializer(torch.empty(self.data.n_users, self.input_dim)).to(device)),
            'item_emb': nn.Parameter(initializer(torch.empty(self.data.n_items, self.input_dim)).to(device))
        })
        return embedding_dict
    
    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            init.normal_(m.weight, std=0.01)
            if m.bias is not None:
                init.zeros_(m.bias)
        # elif isinstance(m, nn.Parameter):
        #     init.normal_(m.data, std=0.01)

    def forward(self, perturbed_adj=None):
        uEmbed = self.embedding_dict['user_emb']
        iEmbed = self.embedding_dict['item_emb']

        uEmbed = self.act(self.fc_u(uEmbed))
        iEmbed = self.act(self.fc_i(iEmbed))
        
        embeds = torch.cat([uEmbed, iEmbed], 0)
        all_embeddings = [embeds]
        
        if self.use_drop_edge:
            hyperULat = self.hgnn_u(uEmbed, self.edge_index_aug, hyperedge_attr=iEmbed)
            hyperILat = self.hgnn_i(iEmbed, self.edge_index_aug_t, hyperedge_attr=uEmbed)
        else:
            hyperULat = self.hgnn_u(uEmbed, self.edge_index, hyperedge_attr=iEmbed)
            hyperILat = self.hgnn_i(iEmbed, self.edge_index_t, hyperedge_attr=uEmbed)
        ego_embeddings = torch.cat([hyperULat, hyperILat], dim=0)
        all_embeddings += [ego_embeddings]
        
        all_embeddings = torch.stack(all_embeddings, dim=1)
        all_embeddings = torch.mean(all_embeddings, dim=1)
        user_all_embeddings = all_embeddings[:self.data.n_users]
        item_all_embeddings = all_embeddings[self.data.n_users:]
        return user_all_embeddings, item_all_embeddings 
    
    def cal_cl_loss(self, idxs, perturbed_mat1, perturbed_mat2):
        if type(idxs[0]) is not list:
            u_idx = torch.unique(idxs[0])
        else:
            u_idx = torch.unique(torch.Tensor(idxs[0]).to(device).type(torch.long))
        if type(idxs[1]) is not list:
            i_idx = torch.unique(idxs[1])
        else:
            i_idx = torch.unique(torch.Tensor(idxs[1]).to(device).type(torch.long))
        user_view_1, item_view_1 = self.forward(perturbed_mat1)
        user_view_2, item_view_2 = self.forward(perturbed_mat2)
        view1 = torch.cat((user_view_1[u_idx],item_view_1[i_idx]),0)
        view2 = torch.cat((user_view_2[u_idx],item_view_2[i_idx]),0)
        return InfoNCE(view1,view2,self.temp)


## Loss

In [7]:
def calculate_loss(anchor_emb, pos_emb, neg_emb, reg):
    calc_reg_loss = EmbLoss()
    rec_loss = bpr_loss(anchor_emb, pos_emb, neg_emb)
    reg_loss = reg * calc_reg_loss(anchor_emb, pos_emb, neg_emb)
    return rec_loss, reg_loss 

def calculate_kg_loss(anchor_emb, pos_emb, neg_emb, reg):
    calc_reg_loss = EmbLoss()
    rec_loss = triplet_loss(anchor_emb, pos_emb, neg_emb)
    reg_loss = reg * calc_reg_loss(anchor_emb, pos_emb, neg_emb)
    return rec_loss, reg_loss 


In [8]:
def predict(u, rec, user_emb, item_emb):
    user_id  = rec.data.get_user_id(u)
    score = torch.matmul(user_emb[user_id], item_emb.transpose(0, 1))
    return score.cpu().numpy()  

## Train

In [9]:
def train(train_model, data):
    lst_train_losses = []
    lst_rec_losses = []
    lst_reg_losses = []
    lst_cl_losses = []
    lst_performances = []
    recall_list = []
    
    for ep in range(maxEpoch):
        train_losses = []
        rec_losses = []
        reg_losses = []
        cl_losses = []
        
        # dropped_adj1 = train_model.graph_reconstruction()
        # dropped_adj2 = train_model.graph_reconstruction()
        
        train_model.train()
        
        for n, batch in enumerate(next_batch_pairwise(rec.data, batchSize)):
            user_idx, pos_idx, neg_idx = batch
            user_emb, item_emb = train_model()
            anchor_emb = user_emb[user_idx]
            pos_emb = item_emb[pos_idx]
            neg_emb = item_emb[neg_idx]
            
            rec_loss, reg_loss = calculate_loss(anchor_emb, pos_emb, neg_emb, reg)
            cl_loss = 0 
            # cl_loss = cl_rate * train_model.cal_cl_loss([user_idx, pos_idx], dropped_adj1, dropped_adj2)
            batch_loss = rec_loss + reg_loss 
            train_losses.append(batch_loss.item())
            rec_losses.append(rec_loss.item())
            reg_losses.append(reg_loss.item())
            # cl_losses.append(cl_loss.item())
            
            optimizer.zero_grad()
            batch_loss.backward()
            torch.nn.utils.clip_grad_norm_(train_model.parameters(), 4)
            optimizer.step()

        batch_train_loss = np.mean(train_losses)
        scheduler.step(batch_train_loss)
                
        train_loss = np.mean(train_losses)
        rec_loss = np.mean(rec_losses)
        reg_loss = np.mean(reg_losses)
        # cl_loss =  np.mean(cl_losses)
        cl_loss = 0
        
        lst_train_losses.append([ep, train_loss])
        lst_rec_losses.append([ep,rec_loss])
        lst_reg_losses.append([ep, reg_loss])
        lst_cl_losses.append([ep, cl_loss])

        # Evaluation
        train_model.eval()
        with torch.no_grad():
            user_emb, item_emb = train_model()
            data_ep = rec.fast_evaluation(train_model, ep, user_emb, item_emb)

            cur_recall =  float(data_ep[2].split(':')[1])
            recall_list.append(cur_recall)
            best_recall, should_stop = early_stopping(recall_list, 100)
        if should_stop:
            break

        lst_performances.append(data_ep)

    rec.save_loss(lst_train_losses, lst_rec_losses, lst_reg_losses, lst_cl_losses)
    rec.save_perfomance_training(lst_performances)
    user_emb, item_emb = rec.best_user_emb, rec.best_item_emb
    return user_emb, item_emb

## Test

In [10]:
def test(rec, user_emb, item_emb):
    def process_bar(num, total):
        rate = float(num) / total
        ratenum = int(50 * rate)
        r = '\rProgress: [{}{}]{}%'.format('+' * ratenum, ' ' * (50 - ratenum), ratenum*2)
        sys.stdout.write(r)
        sys.stdout.flush()

    # predict
    rec_list = {}
    user_count = len(rec.data.test_set)
    for i, user in enumerate(rec.data.test_set):
        # s_find_candidates = time.time()
        candidates = predict(user, rec, user_emb, item_emb)
        # e_find_candidates = time.time()
        # print("Calculate candidates time: %f s" % (e_find_candidates - s_find_candidates))
        # predictedItems = denormalize(predictedItems, self.data.rScale[-1], self.data.rScale[0])
        rated_list, li = rec.data.user_rated(user)
        for item in rated_list:
            candidates[rec.data.item[item]] = -10e8

        # s_find_k_largest = time.time()
        ids, scores = find_k_largest(rec.max_N, candidates)
        # e_find_k_largest = time.time()
        # print("Find k largest candidates: %f s" % (e_find_k_largest - s_find_k_largest))
        item_names = [rec.data.id2item[iid] for iid in ids]
        rec_list[user] = list(zip(item_names, scores))
        if i % 1000 == 0:
            process_bar(i, user_count)
    process_bar(user_count, user_count)
    print('')
    rec.evaluate(rec_list)

## Main

In [11]:
model = 'HGNN'
if model not in ['HGNN', 'LightGCN']:
    print("No model found.")
config = ModelConf('./conf/' + model + '.conf')

dataset = ['lastfm']
batchSize = 2048
maxEpoch = 1000
lRates = [0.01]
lrDecays = [0.7]
regs = [0.1]
hyperDims = [128]
inputDims = [32]
ps = [0.1, 0.2, 0.3, 0.4]
dropRates = [0.1, 0.2, 0.3, 0.4]
nLayers = [2]
nHeads = [2]
nSelfAtt = [1]
clRate = [0.01]
augType = [1]
temp = [0.3]
w_decays= [0]
optimizers = ['adamw']
attention_modes = ['edge']
concats = [ False ]
use_norms = [True]
use_skip_connections = [True]
use_drop_edges = [True]
hyperparameters = [dataset, lRates, lrDecays, regs, hyperDims, inputDims, ps, dropRates, nLayers, nHeads, nSelfAtt, clRate, augType, temp, w_decays, optimizers, attention_modes, concats, use_norms, use_skip_connections, use_drop_edges]

In [ ]:
for params in product(*hyperparameters):
    dataset, lr, lr_decay, reg, hyper_dim, input_dim, prob, drop_rate, n_layers, n_heads, n_self_att, cl_rate, aug_type, temp, w_decay, opt, attention_mode, concat, use_norm, use_skip_connection, use_drop_edge= params
    print(params)
    args = {
        'lr': lr,
        'lr_decay': lr_decay,
        'reg': reg,
        'hyper_dim': hyper_dim,
        'input_dim': input_dim,
        'p': prob,
        'drop_rate': drop_rate,
        'n_layers': n_layers,
        'input_dim': input_dim,
        'hyper_dim': hyper_dim,
        'n_heads': n_heads,
        'n_self_att': n_self_att,
        'dataset': dataset,
        'cl_rate': cl_rate,
        'aug_type': aug_type,
        'temp': temp,
        'weight_decay': w_decay,
        'optimizer': opt,
        'attention_mode': attention_mode,
        'concat': concat,
        'use_norm': use_norm,
        'use_skip_connection': use_skip_connection,
        'use_drop_edge': use_drop_edge
    }
    training_data = FileIO.load_data_set('./dataset/' + dataset + '/' +config['training.set'], config['model.type'])
    test_data = FileIO.load_data_set('./dataset/' + dataset + '/'  +config['test.set'], config['model.type'])
    knowledge_set = FileIO.load_kg_data('./dataset/' + dataset +'/'+ dataset +'.kg')
    data = Interaction(config, training_data, test_data)

    rec = GraphRecommender(config, data, data, knowledge_set, **args)
    train_model = Model(rec.config, rec.data, args).to(device)
    if opt == 'adam':
        optimizer  = torch.optim.Adam(train_model.parameters(), lr=lr)
    elif opt =='adamw': 
        optimizer  = torch.optim.AdamW(train_model.parameters(), lr=lr)
    elif opt == 'sgd':
        optimizer = torch.optim.SGD(
                train_model.parameters(), lr=lr, momentum=0.9, weight_decay=w_decay, nesterov=True
            )
    scheduler = ReduceLROnPlateau(optimizer, 'min', factor=lr_decay, patience=10)
    user_emb, item_emb = train(train_model, rec.data)
    test(rec, user_emb, item_emb)

('lastfm', 0.01, 0.7, 0.1, 128, 32, 0.1, 0.1, 2, 2, 1, 0.01, 1, 0.3, 0, 'adamw', 'edge', False, True, True, True)
Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 3.312439 s
Measure time: 0.014646 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epoch: 1, Hit Ratio:0.06576  |  Precision:0.0405  |  Recall:0.06515  |  NDCG:0.05908
*Best Performance* 
Epoch:fast_evaluation 1, Hit Ratio:0.06576  |  Precision:0.0405  |  Recall:0.06515  |  NDCG:0.05908
------------------------------------------------------------------------------------------------------------------------
Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.203120 s
Measure time: 0.014946 s
---------------------------------------------------------------------------------